In [4]:
import pandas as pd
import re

# File paths and headers
file_base = "Data 8199 2391 5_2_2025 08_28_33 "
file_paths = [f"{file_base}{i}.xlsx" for i in range(1, 5)]
header_rows = [68, 68, 68, 74]  # header row indices for each file

long_data = []

for path, header_row in zip(file_paths, header_rows):
    df = pd.read_excel(path, engine='openpyxl', header=header_row)

    # Clean columns
    df.columns = df.columns.map(str)
    df = df.loc[:, ~df.columns.str.contains("Alarm|Unnamed", na=False)]
    if "Scan" in df.columns:
        df = df.drop(columns="Scan")

    # Find time column and convert to HH:MM
    time_col = [col for col in df.columns if 'time' in col.lower()][0]
    df[time_col] = pd.to_datetime(df[time_col], format="%m/%d/%Y %H:%M:%S:%f", errors='coerce')
    df[time_col] = df[time_col].dt.strftime("%H:%M")

    # Melt to long format
    df_melted = df.melt(id_vars=[time_col], var_name="SensorFull", value_name="Value")

    # Extract clean sensor name (from <...>)
    df_melted["Sensor"] = df_melted["SensorFull"].str.extract(r"<(.*?)>")
    df_melted = df_melted.dropna(subset=["Sensor"])

    df_melted = df_melted[[time_col, "Sensor", "Value"]]
    df_melted.columns = ["Time", "Sensor", "Value"]

    long_data.append(df_melted)

# Combine all sensor data
df_all_long = pd.concat(long_data, ignore_index=True)

# Pivot to wide format: Time | Sensor1 | Sensor2 | ...
df_wide = df_all_long.pivot_table(index="Time", columns="Sensor", values="Value")

# Optional: reset index to make Time a column again
df_wide.reset_index(inplace=True)

# Save to CSV
df_wide.to_csv("CELLS_surface_temperature_wide_clean.csv", index=False)
print("✅ Wide-format CSV saved as 'CELLS_surface_temperature_wide_clean.csv'")


✅ Wide-format CSV saved as 'CELLS_surface_temperature_wide_clean.csv'


In [5]:
# --- Step 1: Load the cleaned wide-format CSV file ---
df = pd.read_csv("CELLS_surface_temperature_wide_clean.csv")

# Extract all column names except 'Time'
csv_sensor_set = set(df.columns) - {"Time"}

# --- Step 2: Paste in the sensor names from all 4 Excel files (manually copied from your lists above) ---

# Sensors from file 1
file1_sensors = [
    "1_ET1", "1_ET2", "1_ET9", "1_ET3", "1_ET4", "1_ET5", "1_ET8", "1_ET6", "1_ET7",
    "1_WT1", "1_WT2", "1_WT3", "1_WT4", "1_WT5", "1_WT6", "1_WT7", "1_WT8", "1_WT9",
    "1_CT1", "1_CT2", "1_FR1", "1_FR2", "1_FR3", "1_FR4", "1_FR9", "1_FR10", "1_FR11", "1_FR5", "1_FR6", "1_FR7", "1_FR8",
    "1_CT3", "1_CT4", "1_CT5", "1_CT6", "1_CT7", "1_CT8", "1_CT9", "1_CT10", "1_CT11",
    "1_NT1", "1_NT2", "1_NT3", "1_NT4", "1_NT5", "1_NT6", "1_NT7", "1_NT8",
    "1_ST1", "1_ST2", "1_ST3", "1_ST4", "1_SH1", "1_ST6", "1_ST7", "1_ST8",
    "1_VO", "1_VI", "2_VO", "2_VI"
]

# Sensors from file 2
file2_sensors = [
    "2_FT1", "2_FT2", "2_FT3", "2_FT4", "2_FT5", "2_FT6", "2_FT7", "2_FT8", "2_FT9", "2_FT10", "2_FT11", "2_FT12", "2_FT13",
    "0_WT7", "0_WT8", "0_WT9",
    "2_CH1", "2_CH2", "2_CH3", "2_CH4",
    "2_EH1", "2_NH1", "2_WH1", "2_WH2", "2_WH3", "2_SH1",
    "2_FH1", "2_FH2", "1_FH1", "1_FH2",
    "1_EH1", "1_EH2", "1_EH3", "1_WR1", "1_NH1", "1_ST5",
    "1_CH1", "1_CH2", "1_CH3", "1_CH4",
    "1_NT9", "1_FR12", "1_FR13", "1_WT10", "1_ST9",
    "1_ER1", "1_ER2", "1_CT12", "1_CT14", "1_CT15", "1_CT16",
    "0_ET1", "0_ET2", "0_ET3", "0_ET4", "0_ET5", "0_ET6", "0_ET7", "0_ET8", "0_ET9"
]

# Sensors from file 3
file3_sensors = [
    "2_ET1", "2_ET2", "2_ET3", "2_ET4", "2_ET5", "2_ET6", "2_ET7", "2_ET8", "2_ET9", "2_ET10",
    "2_NT1", "2_NT2", "2_NT3", "2_NT4", "2_NT5", "2_NT6", "2_NT7", "2_NT8", "2_NT9",
    "0_WT1", "2_WT1", "2_WT2", "2_WT3", "2_WT4", "2_WT5", "2_WT6", "2_WT7", "2_WT8", "2_WT9", "2_WT10", "2_WT11",
    "2_ST1", "2_ST2", "2_ST3", "2_ST4", "2_ST5", "2_ST6", "2_ST7", "2_ST8", "2_ST9",
    "2_CT1", "2_CT2", "2_CT3", "2_CT4", "2_CT5", "2_CT6", "2_CT7", "2_CT8", "2_CT9", "2_CT10", "2_CT11", "2_CT12", "2_CT13", "2_CT14", "2_CT15",
    "0_WT2", "0_WT3", "0_WT4", "0_WT5", "0_WT6"
]

# Sensors from file 4
file4_sensors = [
    "WO1", "WO2", "WO3", "WO4", "WO5", "WO6", "WO7", "WO8", "WO9", "WO10", "WO11", "WO12", "WO13", "WO14", "WO15", "WO16", "WO17", "WO18", "WO19", "WO20",
    "An_out_Temp_W", "An_out_Temp_E",
    "WO21", "WO22", "WO23", "WO24", "WO25",
    "FO1", "FO2", "RO1", "RO2", "RO3", "RO4", "WinO1", "WinO2",
    "EO1", "EO2", "EO3", "EO4", "EO5", "EO6", "EO7", "An_out_Vel_W", "An_out_Vel_E",
    "EO8", "EO9", "EO10", "EO11", "EO12", "EO13", "EO14", "EO15", "EO16", "EO17", "EO18", "EO19", "EO20", "EO21", "EO22", "EO23", "EO24", "EO25",
    "An_in_Vel_W", "An_in_Vel_E"
]


# --- Step 3: Compare ---
all_expected_sensors = set(file1_sensors + file2_sensors + file3_sensors + file4_sensors)
missing_sensors = all_expected_sensors - csv_sensor_set
present_sensors = all_expected_sensors & csv_sensor_set

print(f"✅ Found {len(present_sensors)} / {len(all_expected_sensors)} expected sensors in CSV.")

if missing_sensors:
    print("❌ Missing sensors:")
    for sensor in sorted(missing_sensors):
        print("-", sensor)
else:
    print("🎉 All expected sensors are present!")


✅ Found 244 / 244 expected sensors in CSV.
🎉 All expected sensors are present!
